In [2]:
import yaml
import cv2
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output, Image, display
import time


In [3]:
#import scripts files
from scripts.vision import Vision
from scripts.global_nav import GlobalNav
from scripts.local_nav import LocalNav
from utils.utils import *

In [19]:
# Load the configuration file
with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Controller configuration
angle_tolerance = config['controller']['angle_tolerance']
distance_tolerance = config['controller']['distance_tolerance']
scale_rotation_speed = config['controller']['scale_rotation_speed']
max_rotation_speed = config['controller']['max_rotation_speed']
scale_translation_speed = config['controller']['scale_translation_speed']
min_translation_speed = config['controller']['min_translation_speed']
max_translation_speed = config['controller']['max_translation_speed']

# World configuration
world_width = config['world']['width']
world_height = config['world']['height']

# Thymio configuration
thymio_width = config['thymio']['size']['width']
thymio_length = config['thymio']['size']['length']

# Webcam configuration
webcam_device_id = config['webcam']['device_id']
webcam_matrix = np.array(config['webcam']['matrix'])
webcam_distortion = np.array(config['webcam']['distortion'])
webcam_resolution = config['webcam']['resolution']
webcam_padding = config['webcam']['padding']


In [20]:
vision = Vision(
    device_id=webcam_device_id,
    camera_matrix=webcam_matrix,
    dist_coeffs=webcam_distortion,
    resolution=webcam_resolution,
    padding=webcam_padding,
    world_width=world_width,
    world_height=world_height
)

Vision Initialized


In [25]:
def display_frames(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    _, buffer = cv2.imencode('.jpg', frame_rgb)
    display(Image(data=buffer.tobytes()))
    clear_output(wait=True)

try:
    # Try connecting to the webcam
    print(f"Trying to connect to device {webcam_device_id}...")
    if not vision.connect_webcam():
        print("Could not find webcam on any device ID. Please check connection.")
        exit(1)
    else: 
        print(f"Successfully connected to device {webcam_device_id}")
        
    while True:
        origin_frame, process_frame, _, _ = vision.get_frame()
        
        if process_frame is not None:
            # Display just the process frame
            display_frames(process_frame)
        else:
            display_frames(origin_frame)
        # Add a small delay to control frame rate
        time.sleep(0.1)
        
        # Check if kernel interrupt was requested
        try:
            if get_ipython().kernel.is_interrupted():
                break
        except:
            pass
      
except KeyboardInterrupt:
    print("Streaming stopped")
finally:
    # Release resources
    vision.cleanup_webcam()

Streaming stopped
